In [1]:
import pandas as pd
import os
import mrcfile
import csv
import numpy as np

In [12]:
filepath = "ChainA_simulated_resample_phase2pred.txt"
df = pd.read_csv(filepath, sep=' |,', engine='python')
df.drop(df.columns[3],axis=1,inplace=True)

In [13]:
print(df) # x: 180 - 230, y: 68 - 103, z: 90 - 175 

        180  103   90  1  0.2907393  0.3802863  0.30566594  0.02330846
0       181  103   90  1   0.308517   0.369448    0.292441    0.029594
1       182  103   90  1   0.309514   0.362327    0.299106    0.029052
2       183  103   90  1   0.330668   0.343932    0.290840    0.034560
3       184  103   90  1   0.330855   0.343784    0.290682    0.034679
4       185  103   90  1   0.330946   0.343733    0.290558    0.034762
...     ...  ...  ... ..        ...        ...         ...         ...
541762  226   68  175  2   0.317804   0.250222    0.387394    0.044580
541763  227   68  175  2   0.317466   0.250439    0.387960    0.044135
541764  228   68  175  2   0.294985   0.294741    0.359653    0.050621
541765  229   68  175  2   0.310936   0.297494    0.351076    0.040493
541766  230   68  175  2   0.292954   0.310799    0.349195    0.047052

[541767 rows x 8 columns]


In [14]:
df.to_csv("Input_emd_8097_chainA_resampled.csv", index=False)

In [21]:
def conv_phase_to_mrc1(phasefile,reffile,keyidx,outmapfile):
    if os.path.isfile(outmapfile):
        return True
    INPUT=[]
    with open(phasefile) as f:
        reader = csv.reader(f)
        for row in reader:
            #xyz:[0,3]
            INPUT.append([int(float(row[0])),int(float(row[1])),int(float(row[2])),float(row[keyidx+3])])
    with mrcfile.open(reffile) as mrc:
        nx,ny,nz,nxs,nys,nzs,mx,my,mz = mrc.header.nx,mrc.header.ny,mrc.header.nz,mrc.header.nxstart,mrc.header.nystart,mrc.header.nzstart,mrc.header.mx,mrc.header.my,mrc.header.mz
        orig=mrc.header.origin
        data=mrc.data
        data_new= data #init
        #update data by INPUT
        #mrc_new = mrcfile.new(outmapfile,data=data_new, overwrite=True)
        
        mrc_new = mrcfile.new(outmapfile,overwrite=True)
        
        mrc_new.set_data(np.zeros((nx * 2, ny * 2, nz * 2),dtype=np.float32))
        mrc_new.header.nxstart=nxs
        mrc_new.header.nystart=nys
        mrc_new.header.nzstart=nzs
        
        for x,y,z,prob in INPUT:
            mrc_new.data[z,y,x]=prob
        vsize=mrc_new.voxel_size
        vsize.flags.writeable = True
        mrc_new.voxel_size=vsize         
         #             mrc_new.header.xorg=float(xx[0]) - float(mx)/2.0
        
        mrc_new.update_header_stats()
        
        print("original", mrc.voxel_size)
        mrc.print_header()
        
        print("new", mrc_new.voxel_size)
        mrc_new.print_header()
        
        mrc_new.close()    

In [22]:
structures = [1, 2, 3, 4]
csv_path = "Input_emd_8097_chainA_resampled.csv"
mrc_path = "ChainA_simulated_resample.mrc"
map_name = "emd_8097_chainA"

for i in structures:
    if i == 1:
        name = "_helix.mrc"
    elif i == 2:
        name = "_coil.mrc"
    elif i == 3:
        name = "_strand.mrc"
    else:
        name = "_DNA.mrc"
      
    conv_phase_to_mrc1(csv_path, mrc_path, i, map_name + name)

original (1.0700002, 1.0700002, 1.0700002)
nx              : 221
ny              : 201
nz              : 191
mode            : 2
nxstart         : 0
nystart         : 0
nzstart         : 0
mx              : 221
my              : 201
mz              : 191
cella           : (236.47003, 215.07004, 204.37003)
cellb           : (90., 90., 90.)
mapc            : 1
mapr            : 2
maps            : 3
dmin            : 0.0
dmax            : 0.5139835476875305
dmean           : 0.00250758184120059
ispg            : 0
nsymbt          : 0
extra1          : b'\x00\x00\x00\x00\x00\x00\x00\x00'
exttyp          : b''
nversion        : 0
extra2          : b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
origin  